# Playing with the SVD

Today we're going to play a little bit with the SVD, and see how we can effectively use it for compression.  We're going to work with a small dataset, and then a big one.  First, here's us loading up the foods dataset again.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
fAM=pd.read_csv('foodsAndMovies.csv.gz')
fAM=fAM[['Broccoli','Mushrooms','Beef Tacos','Salads','Black Licorice','Steak','Grilled Chicken','Mayonnaise','Candy Corn','Pulled Pork','Spicy Mustard','Raw Oysters','Bananas','Avocado','Eggs','Olives','Tofu','Cottage Cheese']]
means=fAM.mean()
for col in fAM.columns:
    fAM[col]=fAM[col].fillna(means[col])
fAM.head()

... and turning it into a matrix ...

In [ ]:
foods=fAM.to_numpy()
print(f'Shape of "foods" matrix: {foods.shape}')
foods

OK!  Now, [take the SVD](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html) of the foods matrix.  When you're done, you should have three vectors/matrices.  Print out their shapes.  Those shapes should make sense to you.

Additionally, plot the singular values.

Of course, we know we should be able to remultiply $U\Sigma V^T$ and reproduce our original `foods` matrix.  Write a function you can call to see how accurate this reproduction is.  This is a simple function - it should accept two arguments, `original` and `reproduction`. It should subtract one from the other, and calculate [the Frobenius norm of the result](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html), divided by the number of elements in the matrix (therefore giving you an idea of the average error per element).  Do not hardcode in matrix sizes, as we'll be calling this function on other datasets later in the lab.  It should return the computed error value.

Build a $\Sigma$ matrix from your vector of singular values, and remultiply $U\Sigma V^T$ to reproduce your original matrix.  Call your function, and see what your error is.  If it's not quite small, something is wrong.

We're going to start truncating these matrices; you will find slicing to be the best tool for this.  We know the last columns of $U$ and the last rows of $V^T$ are the least important (ie, the last singular vectors), because they're multiplied by the smallest singular values.  Let's suppose we only want to keep 17 of our 18 singular values/vectors.  On paper, confirm to yourself that if we kept only the first 17 columns of $U$, the top-left 17x17 block of $Sigma$, and the top 17 rows of $V^T$, the product of those three truncated matrices would be the same size as our original matrix.

Then create those matrices, multiply them together to build a reproduction of `foods`, and use your function to tell me the error in that reproduction.

Repeat this process for keeping 1 singular value/vector, then 2, then 3, up through all 18.  Build a line plot of the errors that result.  Naturally the largest errors should be when keeping only one, and the smallest when keeping them all.

Keeping in mind these values are the number of "stars" off the average reproduced rating is from the original dataset, tell me how many features you think are necessary for the reproduction to be a reasonable approximation for this dataset.

Let's do this again with a different dataset.  Run the below (commented for your understanding) code to load the "mnist" dataset, a commonly used image recognition dataset consisting of handwritten digits.  We're going to play with the only the threes.

In [ ]:
#Load the data. pics is the images, labels is a vector indicating what number is depicted in that datapoint
with open('mnist.npy','rb') as f:
    pics=np.load(f)
    labels=np.load(f)
#Each datapoint is a 28x28 array of pixel values. There are 60000 images in the dataset.
print(f'The shape of the pics array is {pics.shape}, the shape of labels is {labels.shape}.')

#These lines find the indices of the pictures of threes, and pulls them out of the dataset of pictures.
indicesOfThrees=np.nonzero(labels==3)[0]
threes=pics[indicesOfThrees,:,:]
print(f'The shape of the threes aray is {threes.shape}, so there are {threes.shape[0]} 3s in the dataset.')
numThrees=threes.shape[0]

#This line reshapes this into a 2D array, where each row is the 28x28=784 pixel values that represent a single image
threes=np.reshape(threes,(numThrees,784))
print(f'After reshaping, the shape of the threes array is {threes.shape}.')

This function will display the first `n` rows as images.  Run this cell to look at the first three 3s.

In [ ]:
import matplotlib.pyplot as plt
def showNums(data,n):
    data=np.reshape(data[:n,:],(n,28,28))
    for i in range(n):
        plt.subplot(n,1,i+1)
        plt.imshow(data[i,:,:],cmap='gray')
    plt.show()
showNums(threes,3)

OK, so this `threes` array is now just an array like our `foods` array.  Presumably, there is significant correlation between these pixel values (columns), as not every pixel is independent (because not every 28x28 image is of a 3).  Calculate the SVD of this matrix, and plot the singular values.

From this graph, what can you tell me about the relative importance of some of these 
singular vectors from others?  Are they all likely very important?

Like you did with the `foods` matrix, for all values from 1 to 784, keep that many singular vectors from your SVD matrices, and calculate the error in the resulting reproduction.  Tell me what this tells you about this dataset.

Fortunately with this dataset, we can not only see the numerical error, we can actually look at how the rows (ie, pictures of 3s) change in the reproduced matrix from the original matrix.

This function will display the first three 3s from both the original matrix (on the left) and the reproduction (on the right).  Run this cell to define the function.

In [ ]:
def showReconstruct(data,rebuilt):
    en,k=data.shape
    data=np.reshape(data,(en,28,28))
    en,k=rebuilt.shape
    rebuilt=np.reshape(rebuilt,(en,28,28))
    plt.subplot(3,2,1)
    plt.imshow(data[0,:,:],cmap='gray')
    plt.subplot(3,2,3)
    plt.imshow(data[1,:,:],cmap='gray')
    plt.subplot(3,2,5)
    plt.imshow(data[2,:,:],cmap='gray')
    plt.subplot(3,2,2)
    plt.imshow(rebuilt[0,:,:],cmap='gray')
    plt.subplot(3,2,4)
    plt.imshow(rebuilt[1,:,:],cmap='gray')
    plt.subplot(3,2,6)
    plt.imshow(rebuilt[2,:,:],cmap='gray')
    plt.show()

Run this function, passing in a reproduction from keeping 700 of the 784 singular values/vectors.

Run this several times below, with different values of singular values/vectors.  If your goal was to reproduce images of 3s with as few singular values/vectors as possible, how many do you think are necessary?